# Mountain Car Using Cross-Entropy Reinforcement Learning

## Imports ##

In [1]:
import gym
import tensorflow as tf
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

## Environment ##

In [2]:
# Create the Mountain Car game environment
env = gym.make('MountainCar-v0')


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/Users/rnieves/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


## Hyperparameters ##

There are several changes done here as compared with the version written for the "CartPole" problem:

* The `state_size` and `action_size` variables were initialized using information provided by the OpenAI Gym environment.
* The batch size used in the cross-entropy method was raised from `25` to `50`.

In [3]:
# Environment parameters
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

hidden_layer_size = 128

batch_size = 50

learning_rate = 0.01

max_episodes = 100

max_steps = 200
percentile = 70

## Neural network ##

In [4]:
class Net:
    def __init__(self, 
                 state_size = state_size, 
                 action_size = action_size, 
                 hidden_layer_size = hidden_layer_size,
                 learning_rate = learning_rate, 
                 name = 'net'):
        
        with tf.variable_scope(name):
        
            ### Prediction part
        
            # Input layer, state s is input
            self.states = tf.placeholder(
                tf.float32, 
                [None, state_size])
            
            # Hidden layer, ReLU activation
            self.hidden_layer = tf.contrib.layers.fully_connected(
                self.states, 
                hidden_layer_size)
            
            # Hidden layer, linear activation, logits
            self.logits = tf.contrib.layers.fully_connected(
                self.hidden_layer, 
                action_size,
                activation_fn = None)
            
            # Output layer, softmax activation yields probability distribution for actions
            self.probabilities = tf.nn.softmax(self.logits)
    
            ### Training part 
    
            # Action a
            self.actions = tf.placeholder(
                tf.int32, 
                [None])
            
            # One-hot encoded action a 
            #
            # encoded_action_vector = [1, 0] if action a = 0
            # encoded_action_vector = [0, 1] if action a = 1
            self.one_hot_actions = tf.one_hot(
                self.actions, 
                action_size)

            # cross entropy
            self.cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(
                logits = self.logits, 
                labels = self.one_hot_actions)
            
            # cost
            self.cost = tf.reduce_mean(self.cross_entropy)
            
            # Optimizer
            self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
            
    # get action chosen according to current probabilistic policy
    def get_action(self, state):
        feed_dict = { self.states : np.array([state]) } 
        probabilities = sess.run(self.probabilities, feed_dict = feed_dict)
        
        return np.random.choice(action_size, p=probabilities[0])
    
    # train based on batch
    def train(self, batch):
        states, actions = zip(*batch)
        states = np.array(states)
        actions = np.array(actions)
        
        feed_dict = {
            self.states : states,
            self.actions : actions
        }
        
        sess.run(self.optimizer, feed_dict = feed_dict)

## Reward Modification
The default reward signal from the "Mountain Car" scenario is too sparse for it to be useful when implementing a solution involving a fully-connected neural network that uses a cross-entropy loss function. Thus, this solution uses a modified reward formula in an attempt to train the neural network to successfully achieve victory. The reward function modification went through a non-trivial evolution, with many schemes being rejected for failing to converge toward a winning solution after a reasonable number of training episodes.

In all of the modified reward functions attempts, a single time step reward of $1,000$ would be awarded if the car reached the flag, perceived by the episode being done while more time steps remained in the episode. This single step reward would let the training algorithm detect a victory condition.

### Attempt 1: Biased Position
The Mountain Car scenario state representation consists of two (2) real numbers: car position and car velocity. Car position seems to be expressed in terms of the car position with respect to a flat X axis located laid horizontally across the scenario environment. Based on experimentation, the effective range of this X axis is $[-1.2, 0.5]$. Such a range exhibits a median value of $-0.85$, and it would be reasonable to assume this coordinate points to the lowest point in the environment's valley. Further observations, however, reveal that the X coordinate for the valley's lowest point is actually $-0.5$. Using this information, the first attempt at reward modification was to simply return the car's X-coordinate value, biased by $0.5$.

$$ R(s) = s_{pos} + 0.5 $$

This reward function exhibited improvement through the first $10,000$ training episodes or so, as observed via test episodes run on the model after every $100$ training episodes. However, after another $10,000$ training episodes, the model lingered around the bottom half of the hill leading to the flag, failing to leverage the far side of the valley for momentum.

### Attempt 2: ReLU'd Biased Position
Continuing the use of the mountain car's X-coordiate biased by $0.5$, the next reward function attempt consisted in doing away with a negative reward (i.e., a penalty) whenever the car crossed over into the far side of the valley. Instead, in the far side of the valley the scenario would not offer a reward at all. In essence the Biased Position reward function would be put through a Rectifying Linear Unit (ReLU). The hope would be that the lack of a penalty would at least remove the apparent prohibition on the use of the far side slope.

$$ R(s) = \max(0.0, (s_{pos} + 0.5)) $$

Unfortunately, this modified reward function exhibited the same behavior as the original Biased Position: the car would not venture into the far side of the valley in order to gain momentum.

### Attempt 3: Squared, ReLU'd Biased Position
The next step involved the incentivization of gaining "altitude" on the slope closest to the flag, by deriving a reward that would grow in a non-linear fashion at higher positions in the near slope. In the far side of the valley, the reward would still be clamped at $0.0$. The intent driving this approach was to incentivize the model to reach higher positions in the slope nearest to the flag. The non-linear function used in this attempt was simply the squaring of the rectified X coordinate.

$$ R(s) = (\max(0.0, (s_{pos} + 0.5)))^2 $$

Unfortunately, since the biased X coordinate range between the lowest point in the valley and the flag range between $[0.0, 1.0]$, the function's slope in this range is "flatter" than the linear range by itself. Only after crossing the $1.0$ value would a the square function being to exhibit a more pronounced slope compared to a linear function. Thus, the model failed to climb very high up the slope nearest to the flag.

### Attempt 4: ReLU'd, Sinusoid Based on Biased Position
The next reward function evolution attempted to leverage the fact that the valley "shape" closely resembled a sinusoid. Perhaps, if the biased X coordinate was used to derive the corresponding result of a sinusoidal function, and that result was used as the basis for the reward, it would provide a stronger incentive to "climb the hill", since the result of this function would grow at a faster clip than the X coordinate alone. The range of rewards in the far side of the hill would remain ReLU'd at $0.0$. The rewards in the near side of the valley would still be mapped to the range $[0.0, 1.0]$, but in a non-linear fashion as defined by $[\sin(0.0), \sin(\pi/2)]$. Thus, the biased X coordinate $s_{bpos} \in [0.0, 1.0]$ needs to be mapped to the range $\theta \in [0.0, \pi/2]$ prior to being fed to the $\sin()$ function.

$$ \frac{s_{bpos}}{1.0 - 0.0} = \frac{\theta}{\pi/2 - 0.0} \\
s_{bpos} = \frac{\theta}{\pi/2} \\
\theta = \frac{\pi s_{bpos}}{2} \\
R(s) = \max(0.0, \sin(\theta))
$$

After this attempt failed, it became obvious that, regardless of the non-linear rewards in the near side of the valley, refraining from providing any reward in the far side of the valley was detrimental to the successful training of the model.

### Attempt 5: "Altitude" Reward
Starting from the ReLU'd sinusoid attempt, if the step reward was at least based on "how high" the model drove the cart, in either direction, perhaps the model would eventually learn to "aim for the fences" and, consequently, reach the flag. Retaining the previous mapping of the biased X coordinate $s_{bpos} \in [0.0, 1.0]$ to the range $\theta \in [0.0, \pi/2]$, a new mapping for the far side of the valley could be derived such that $s_{bpos} \in [-0.7, 0.0]$ maps to $\theta_f \in [\pi/2, 0.0]$.

$$ \frac{s_{bpos}}{0.0 - -0.7} = \frac{\theta_f}{\pi/2 - 0.0} \\
\frac{s_{bpos}}{0.7} = \frac{\theta_f}{\pi/2} \\
\theta_f = \frac{\pi s_{bpos}}{2.0 \times 0.7} \\
\theta_f = \frac{\pi s_{bpos}}{1.4} \\
R(s) = \begin{cases}
    \sin(\theta) & s_{bpos} \geq 0.0 \\
    \sin(\theta_f) & s_{bpos} < 0.0
\end{cases}
$$

The model's actions precipitated by this reward function exhibited more variety with respect to the side of the valley the car ventured in, but it unfortunately encouraged the model to attempt lingering at either side of the valley hills, regardless of how fast it was moving. Disregarding speed also disregarded momentum and, thus, did not yield a solution that achived victory. Sometimes, the model would favor the far side of the valley, completely disregarding the near side. After this attempt, it was obvious that the model needed to be discouraged from lingering at a specific height, reaping a constant reward, and from disregarding the near side of the valley in favor of the far side.

### Attempt 6: Ranked Altitude Reward Based on Location and Speed
The final attempt that, eventually, yielded a model which successfully reached the flag, started from the previous attempt's "altitude" reward approach. The prior approach was modified in two (2) ways:

* The position reward in the far side of the valley would be "muted" by a fixed factor.
* The total reward would be scaled by the car's speed.

The selection of the "muting factor" for the far side was strictly based on empirical observations, and eventually the value $0.6$ yielded success. Also, the reward would be "scaled" by the car's speed (i.e., the car's velocity without regard to direction). The last addition aimed at disincentivizing the lingering behavior observed when using the last reward function. Basically, the model would be rewarded more as the car gained speed. Further, the speed reward in the near side of the valley (closest to the flag) would be increased by a factor of $2.0$, in order to further incentivize fast motion in the valley's near side.

$$
R(s,v) = \begin{cases}
    2.0 \times \sin(\theta) \times \lvert v \rvert & s_{bpos} \geq 0.0 \\
    0.6 \times \sin(\theta_f) \times \lvert v \rvert & s_{bpos} < 0.0
\end{cases}
$$

In [ ]:
import math

def calculate_reward(cart_position, cart_velocity, done, remaining_timesteps):
    reward = cart_position + 0.5
    if reward < 0.0:
        reward = (math.fabs(reward) * math.pi) / 1.4
        reward = math.sin(reward) * 0.6
        reward *= math.fabs(cart_velocity)
    else:
        reward = (reward * math.pi) / 2.0
        reward = math.sin(reward)
        reward *= (2.0 * math.fabs(cart_velocity))

    # Reached the flag? BONUS
    if (remaining_timesteps > 0) and done:
        reward = 1000.0

    return reward

## Training ##

In [5]:
tf.reset_default_graph()
net = Net(name = 'net',
          hidden_layer_size = hidden_layer_size,
          learning_rate = learning_rate)

In [6]:
import random
import bisect
import time


with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    
    start_index = int(max_episodes * percentile / 100)
    test_index = 0
    while True:
        total_reward_list = []
        trajectory_list = []

        for e in np.arange(max_episodes):
            total_reward = 0.0
            trajectory = []
            state = env.reset()
            for s in np.arange(max_steps):
                action = net.get_action(state)
                next_state, reward, done, _ = env.step(action)
                reward = calculate_reward(next_state[0], next_state[1], done, max_steps - (s+1))
                total_reward += reward
                trajectory.append((state, action))
                state = next_state
                if done: break

            index = bisect.bisect(total_reward_list, total_reward)
            total_reward_list.insert(index, total_reward)
            trajectory_list.insert(index, trajectory)

        # keep the elite episodes, that is, throw out the bad ones 
        # train on state action pairs extracted from the elite episodes
        # this code is not optimized, it can be cleaned up 
        state_action_pairs = []
        for trajectory in trajectory_list[start_index:]:
            for state_action_pair in trajectory:
                state_action_pairs.append(state_action_pair)
        # shuffle to avoid correlations between adjacent states
        random.shuffle(state_action_pairs) 
        n = len(state_action_pairs)
        batches = [state_action_pairs[k:k + batch_size] for k in np.arange(0, n, batch_size)]

        for batch in batches:
            net.train(batch)

        # test agent
        state = env.reset()
        env.render()
        time.sleep(0.05)
        total_reward = 0.0
        for s in np.arange(max_steps):
            action = net.get_action(state)
            state, reward, done, _ = env.step(action)
            reward = calculate_reward(state[0], state[1], done, max_steps - (s+1))
            total_reward += reward
            env.render()
            if done: break
        env.close()
        print("Test {:d} Total Reward: {:f}".format(test_index + 1, total_reward))
        if total_reward > 1000.0:
            break
        test_index += 1

Test 1 Total Reward: 0.044052
Test 2 Total Reward: 0.062883
Test 3 Total Reward: 0.274793
Test 4 Total Reward: 0.111460
Test 5 Total Reward: 0.138190
Test 6 Total Reward: 0.149540
Test 7 Total Reward: 0.162236
Test 8 Total Reward: 0.042849
Test 9 Total Reward: 0.187215
Test 10 Total Reward: 0.369068
Test 11 Total Reward: 0.159081
Test 12 Total Reward: 0.030991
Test 13 Total Reward: 0.154917
Test 14 Total Reward: 0.040828
Test 15 Total Reward: 0.124358
Test 16 Total Reward: 0.161737
Test 17 Total Reward: 0.520044
Test 18 Total Reward: 0.077175
Test 19 Total Reward: 0.172716
Test 20 Total Reward: 0.277317
Test 21 Total Reward: 0.117083
Test 22 Total Reward: 0.297293
Test 23 Total Reward: 0.477915
Test 24 Total Reward: 0.142265
Test 25 Total Reward: 1.082097
Test 26 Total Reward: 0.723146
Test 27 Total Reward: 0.804612
Test 28 Total Reward: 0.687000
Test 29 Total Reward: 0.127395
Test 30 Total Reward: 0.945647
Test 31 Total Reward: 0.140894
Test 32 Total Reward: 1.256090
Test 33 Total Rew